In [1]:
def makeIdeal(R, rep):
    #Generates the ideal of the algebraic representation rep over R
    variables = list(R.variable_names())
    Field = R.base()
    n = len(rep)
    Yvars=['y%d' % i for i in range(n)]
    Zvars=['z%d' % i for i in range(n)]
    G = PolynomialRing(Field,variables + Yvars + Zvars)
    F = G.fraction_field()
    
    denominators=[]
    X = []
    for i in range(len(rep)):
        if F(rep[i]).denominator() != 1:
            denominators.append(G(F(rep[i]).denominator())*G(Zvars[i]) - 1)
            X.append(G(F(rep[i]).numerator()) * G(Zvars[i]) - G(Yvars[i]))
        else:
            X.append( G(rep[i]) - G(Yvars[i]))
    id=G.ideal(X + denominators)
    elid=id.elimination_ideal([G(v) for v in variables] + [G(v) for v in Zvars])
    R = PolynomialRing(Field,Yvars);
    phi=G.hom([0 for _ in variables] + [R(v) for v in Yvars] + [0 for _ in Zvars],R)
    return phi(elid)

def generateCircuitValuation(ideal):
    #Computes the circuit valuation of an algebraic representation
    elid = ideal
    R = elid.ring()
    n = R.ngens()
    Yvars = R.gens()
    coveredAll = False
    circuits = []
    circuitPolys = []
    E = range(n);
    for cl in range(1,n+1):
        if coveredAll:
            break
        coveredAll = True
        for ss in Subsets(n,cl):
            s=frozenset([i-1 for i in ss])
            checkss = True

            for C in circuits:
                if C.issubset(s):
                    checkss = False
                    break

            if checkss:
                coveredAll = False
                Cvars = [Yvars[i-1] for i in ss]
                nonCvars=[v for v in Yvars if v not in Cvars]
                cId = elid.elimination_ideal([R(v) for v in nonCvars])
                if cId.ngens() == 1:
                    if cId.gens()[0] != 0:
                        circuits.append(s)
                        circuitPolys.append(cId.radical().gens()[0])
    circuitValuation = {circuits[i] : circuitValue(circuitPolys[i],n) for i in range(len(circuits))}
    circuitPolynomials = {circuits[i] : circuitPolys[i] for i in range(len(circuits))}
    return circuitValuation, Matroid(circuits=circuits), circuitPolynomials
            
def circuitValue(cpoly,n):
    char = cpoly.base_ring().base().characteristic()
    exponents = cpoly.exponents()
    return [min([valuation(ex[i],char) for ex in exponents]) for i in range(n)]
    
def cval_to_valuation(circuitValuation):
    #Turns a circuit valuation into a basis valuation as in Murota/Tamura
    circuits = [c for c in circuitValuation]
    M = Matroid(circuits=circuits)
    B0 = M.basis()
    g = Graph([tuple(pair) for pair in Subsets([b for b in M.bases()],2) if isNeighbor(pair[0],pair[1])])
    shortestPaths=g.shortest_paths(B0)
    val = {}
    val[B0] = 0
    for B in M.bases():
        if B != B0:
            val[B] = val[B0]
            basisPath = shortestPaths[B]
            for i in range(len(basisPath)-1):
                Bi = basisPath[i]
                Bi1 = basisPath[i+1]
                e = set(Bi.difference(Bi1)).pop()
                f = set(Bi1.difference(Bi)).pop()
                C = M.fundamental_circuit(Bi,f)
                val[B] += circuitValuation[C][e] - circuitValuation[C][f]
    return val

def Lindstrom(I):
    #Computes the Lindstrom valuation of the algebraic representation of the ideal I
    cval, M, cp = generateCircuitValuation(I)
    return M, cval_to_valuation(cval)

def isNeighbor(B1,B2):
    return len(B1.difference(B2))==1

In [2]:
from sage.matroids.advanced import *
def centralPoints(val,M = None, dic=True, integral=True):
    #Computes the central points of M^val
    lp = MixedIntegerLinearProgram()
    alpha = lp.new_variable()
    if not M:
        M = BasisMatroid(Matroid(bases = [B for B in val]))
    E = M.groundset()
    
    #fix the order of the variables in lp by adding dummy constraints
    for e in E:
        lp.add_constraint(alpha[e]==0)
        
    #add the constraints
    tau = {}
    for e in E:
        tau[e] = {B:1*(e in B) for B in val}
    rho = {}
    for B in val:
        rho[B] = val[B] - sum([alpha[e]*tau[e][B] for e in E])
        lp.add_constraint(rho[B] >= 0)
    
    #remove dummy constraints:
    lp.remove_constraints(range(len(E)))
    
    #the central points are the vertices of the polyhedron of the linear program
    pts = lp.polyhedron().vertices_list()
    QQpts = [[QQ(v) for v in pt] for pt in pts]
    
    Ei = {}
    i = 0
    for e in E:
        Ei[e] = i
        i += 1
    
    #normalize points: smallest value 0 in each component
    normPts = []
    normPtsDict = []
    comp = [J for J in M.components()]
    indices = {J:sorted([Ei[j] for j in J]) for J in comp}
    for pt in QQpts:
        mins = {}
        for J in comp:
            minJ = min([pt[j] for j in indices[J]])
            for e in J:
                mins[e] = minJ
        
        #when the input valuation is integral, the central points are integral;
        #we round to counteract machine errors
        if integral:
            normPts.append([round(pt[Ei[e]] - mins[e]) for e in E])
            normPtsDict.append({e:round(pt[Ei[e]] - mins[e]) for e in E})
        else:
            normPts.append([pt[Ei[e]] - mins[e] for e in E])
            normPtsDict.append({e:pt[Ei[e]] - mins[e] for e in E})
    
    if dic:
        return normPtsDict
    else:
        return normPts
        
        
def Balpha(M, val, alpha):
    #Computes B^val_alpha
    val2={ b: sum([alpha[i] for i in b])-val[b] for b in val}
    m = max(val2.values())
    Balpha=[b for b in val.keys() if val2[b]==m]
    return Balpha


def Malpha(M, val, alpha):
    #Computes M^val_alpha
    E= M.groundset()
    B=Balpha(M,val, alpha)
    return BasisMatroid(groundset=E, bases=B)

In [51]:
def FFlockPoint(I,alpha2,pt):
    #input: an ideal I
    #       alpha2 in Z^E
    #       a general point pt of V(I)
    #output: (-m 1)V_alpha2 and -m, where m = max(alpha2)
    
    #pick alpha nonpositive, so that the Frobenius action on pt is nonnegative:
    m = max(alpha2)
    alpha = [a-m for a in alpha2]
    
    R = I.ring()
    F = R.base()
    p = F.characteristic()
    X = list(R.variable_names())
    Y = [x + 'a' for x in X]
    Ra = PolynomialRing(F,X+Y)
    twists = []
    for i in range(len(alpha)):
        k = alpha[i]
        twists.append(Ra(X[i])^(p^(-k)) - Ra(Y[i]))
    
    Ia = Ra.ideal([Ra(g) for g in I.gens()] + twists)
    eI = Ia.elimination_ideal([Ra(x) for x in X])
    R2 = PolynomialRing(F,Y)
    A = AffineSpace(R2)
    S = A.subscheme(eI)
    Si = S.irreducible_components()[0]
    mat = Si.Jacobian_matrix()
    pta = [pt[i]^(p^(-alpha[i])) for i in range(len(alpha))]
    return mat(pta).right_kernel_matrix(), -m
    

import random
def findGeneralPoint(I,F,iters=1000000,includeZero=True):
    #Randomly tries to find a general point of V(I) over a finite field F
    n = len(I.ring().variable_names())
    pts = [pt for pt in F]
    if not includeZero:
        pts.remove(0)
    
    M, val = Lindstrom(I)
    
    cpd = centralPoints(val, forceIntegral=True)
    cp = [[pt[i] for i in pt] for pt in cpd]
    
    N={}
    for alpha in cpd:
        N[tuple([alpha[i] for i in alpha])] = Malpha(M,val,alpha)
    
    it = 0
    
    while it < iters:
        it += 1
        v = [random.choice(pts) for _ in range(n)]
        
        #check if v lies in the variety
        inI = True
        for g in I.gens():
            if g(v) != 0:
                inI = False
                break
        if not inI:
            continue
            
        #check if v is general
        goodV = True
        for alpha in cp:
            A, mm = FFlockPoint(I,alpha,v)
            if not Matroid(matrix=A).is_isomorphic(N[tuple(alpha)]):
                goodV = False
                break
        if goodV:
            return v
    print "Not found in "+ str(iters) +" tries."

Examples:

In [4]:
F2=GF(2)
F8= GF(8)
z3 = F8.gen()
R.<x,y,z> = PolynomialRing(F2)

#An algebraic representation of Non-Pappus:
NonPappusRep=[x,x+y,y,x+y+x*z/(x+y),z,x+y+y*z/(x+y),x*z,x*y+x*y*z/(x+y),y*z]
NonPappusGenPt = [z3, z3^2 + 1, z3^2 + z3 + 1, z3^2, z3^2 + z3 + 1, z3 + 1, z3^2 + 1, z3, z3 + 1]

In [22]:
I = makeIdeal(R,NonPappusRep)
M, val = Lindstrom(I)
cp = centralPoints(val,M,dic=False)
#The central points of the Frobenius flock of NonPappusRep.
[[alpha, FFlockPoint(I,alpha,NonPappusGenPt)] for alpha in cp]

[[[0, 0, 0, 0, 0, 0, 1, 1, 1],
  (
[            1        z3 + 1            z3          z3^2            z3     z3^2 + z3             0             0 z3^2 + z3 + 1]
[            0             0             0             0             0             0             1             0 z3^2 + z3 + 1]
[            0             0             0             0             0             0             0             1            z3],

-1
)],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0],
  (
[            1             0             1             0     z3^2 + z3     z3^2 + z3             0          z3^2        z3 + 1]
[            0             1             1             0            z3            z3          z3^2          z3^2            z3]
[            0             0             0             1 z3^2 + z3 + 1     z3^2 + z3      z3^2 + 1 z3^2 + z3 + 1        z3 + 1],

0
)]]

In [29]:
F4=GF(4)
L = F4.gen()
S.<a,b,c> = PolynomialRing(F4)

#Another algebraic representation of Non-Pappus:
NonPappusRep2=[a^2+b,a,a+b,b+c,b+L*c,c,(L-1)*a^2+L*b+L*c,a^2+b+c-c^2,L*c-a]
NonPappus2GenPt = [0,0,0,0,0,0,0,0,0]

In [30]:
I = makeIdeal(S,NonPappusRep2)
M, val = Lindstrom(I)
cp = centralPoints(val,M,dic=False)

#The central points of the Frobenius flock of NonPappusRep2.
[[alpha, FFlockPoint(I,alpha,NonPappus2GenPt)] for alpha in cp]

[[[0, 0, 0, 1, 0, 0, 1, 1, 0],
  (
[     1      0      1      0     z2      1      0      1 z2 + 1]    
[     0      1      1      0      0      0 z2 + 1      1      1]    
[     0      0      0      1      0      0     z2      1      0], -1
)],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0],
  (
[     1      0      1      0 z2 + 1      1      0      0     z2]   
[     0      1      1      0      0      0      0      0      1]   
[     0      0      0      1     z2      1     z2      1     z2], 0
)],
 [[1, 0, 0, 1, 1, 1, 1, 1, 0],
  (
[     1      0      0      0 z2 + 1      1      0      0      0]    
[     0      1      1      0 z2 + 1      1 z2 + 1      1      1]    
[     0      0      0      1     z2      1     z2      1      0], -1
)]]

In [75]:
T.<x,y,z,u> = PolynomialRing(F4)

#An algebraic representation of a matroid by Lindstrom
grid=[x+y+z+u,x^2+y+z^2+u,y,x^2+y,z,u,L*(x+y)+z+u,L*x+z]

In [76]:
I = makeIdeal(T,grid)
M, val = Lindstrom(I)
cp = centralPoints(val,M,dic=False)

In [77]:
genpt = findGeneralPoint(I,GF(2))

In [78]:
[[alpha, FFlockPoint(I,alpha,genpt)] for alpha in cp]

[[[0, 0, 0, 0, 0, 0, 0, 0],
  (
[     1      0      0      0      0      0     z2     z2]   
[     0      1      0      0      0      1 z2 + 1     z2]   
[     0      0      1      1      0      1 z2 + 1      0]   
[     0      0      0      0      1      0 z2 + 1 z2 + 1], 0
)],
 [[0, 1, 1, 1, 0, 1, 0, 0],
  (
[ 1  0  0  0  1  1  1  1]    
[ 0  1  0  0  0  1  0  0]    
[ 0  0  1  0  1  1 z2 z2]    
[ 0  0  0  1  1  0 z2 z2], -1
)],
 [[0, 0, 1, 1, 0, 0, 0, 0],
  (
[ 1  0  0  0  1  0  1  1]    
[ 0  1  0  0  1  1  0  1]    
[ 0  0  1  0  1  0 z2 z2]    
[ 0  0  0  1  1  0 z2 z2], -1
)],
 [[0, 1, 1, 1, 0, 1, 1, 1],
  (
[     1      0      0 z2 + 1     z2      1      0      1]    
[     0      1      0      0      0      1      0      1]    
[     0      0      1      1      0      1      0 z2 + 1]    
[     0      0      0      0      0      0      1      1], -1
)]]

In [70]:
U.<x,y,z,u> = PolynomialRing(F4)

#An algebraic representation of another matroid by Lindstrom
cube=[x^2+y+z^2+u,y,x^2+y,z,L*(x+y)+z+u,L*x+z,y+u,z+u]

In [71]:
I = makeIdeal(U,cube)
M, val = Lindstrom(I)
cp = centralPoints(val,M,dic=False)

In [72]:
genpt = findGeneralPoint(I,GF(2))

In [73]:
[[alpha, FFlockPoint(I,alpha,genpt)] for alpha in cp]

[[[1, 0, 0, 0, 0, 0, 1, 0],
  (
[     1      0      0      0      0      0      1      0]    
[     0      1      1      0      0     z2 z2 + 1      1]    
[     0      0      0      1      0      0 z2 + 1      1]    
[     0      0      0      0      1      1     z2      0], -1
)],
 [[0, 0, 0, 0, 0, 0, 0, 0],
  (
[     1      0      0      0      0      1      1      1]   
[     0      1      1      0      0 z2 + 1      0      1]   
[     0      0      0      1      0      0      0      1]   
[     0      0      0      0      1      1      0      0], 0
)],
 [[1, 1, 1, 0, 1, 1, 1, 0],
  (
[     1      0      0      0      0      1      1      0]    
[     0      1      0 z2 + 1      0      1     z2 z2 + 1]    
[     0      0      1 z2 + 1      0     z2     z2 z2 + 1]    
[     0      0      0      0      1      1      0      0], -1
)],
 [[1, 1, 1, 0, 0, 0, 1, 0],
  (
[     1      0      0      0      0      0      1      0]    
[     0      1      0      0 z2 + 1 z2 + 1      1      0] 